# Segmenting and Clustering Neighborhoods in Toronto
## Week 3 Peer Graded Assignment

In [1]:
import pandas as pd
import html5lib
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim
import folium
import requests 
from pandas.io.json import json_normalize
import json

/usr/local/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


### Read Given Wiki Page, Search For all Tables and condition to get the table with our required fields

In [11]:
tables= pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
headings=['Postcode','Borough','Neighbourhood']
for table in tables:
    current_headings = table.columns.values[:4]
    if len(current_headings) != len(headings):
        continue
    if all(current_headings == headings):
        break
table.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Convert the table to csv and Call it to a data frame

In [6]:
table[headings].to_csv('data.csv', sep=',', header=True, index=False)

In [7]:
df=pd.read_csv('data.csv')
df.shape

(288, 3)

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing rows with value of Borough = 'Not assigned'

In [7]:
df = df[df.Borough != 'Not assigned']

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Merging rows with same Postcode

In [9]:
aggregation_functions = {'Borough': 'first', 'Neighbourhood': ','.join}
df2 = df.groupby(df['Postcode']).aggregate(aggregation_functions).reset_index()

In [10]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Copying Values 'Borough' to 'Neighbourhood',for rows which having 'Neighbourhood'=='Not Assigned'

In [11]:
df2.loc[df2['Neighbourhood']=='Not assigned','Neighbourhood']=df2.Borough

#### M7A was the only one remaining 'Not assigned' Neighbourhood column. Checking it's current State

In [12]:
df2[df2.Postcode == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [13]:
df2.shape

(103, 3)

In [14]:
df2

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


## Adding Geo Location to the Existing Data Frame

In [15]:
geo_loc=pd.read_csv('https://cocl.us/Geospatial_data')

In [16]:
geo_loc=geo_loc.rename(columns={'Postal Code':'Postcode'})

In [28]:
df_geo=pd.merge(left=df2,right=geo_loc, on='Postcode')
temp_df=df_geo
df_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Find coordinates of Toronto to set the Folium Map 

In [43]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [54]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Slice Data containing only toronto

Checking for 'Borough' for possible values

In [69]:
df_geo['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

Slicing data which has the keyword 'Toronto' in Borough to another dataframe

In [130]:
df_toronto=df_geo[df_geo['Borough'].str.contains(' Toronto')]
df_toronto.shape

(38, 5)

In [140]:
address = 'Central Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [78]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df_geo['Latitude'], df_geo['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [171]:
CLIENT_ID = 'WCBNIU4Q5D3ZFWMLJO3EAYHIAKD2IKEJ0ZJDOJTWRN4EKOMW' # your Foursquare ID
CLIENT_SECRET = 'A2NPSQAHQIWHTVV4ZLVDSDTXEBIVCZIN551WXPHEEZLF15IB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WCBNIU4Q5D3ZFWMLJO3EAYHIAKD2IKEJ0ZJDOJTWRN4EKOMW
CLIENT_SECRET:A2NPSQAHQIWHTVV4ZLVDSDTXEBIVCZIN551WXPHEEZLF15IB


In [172]:
df_toronto=df_toronto.reset_index()


ValueError: cannot insert level_0, already exists

In [143]:
df_toronto.loc[0, 'Neighbourhood']

'The Beaches'

In [144]:
neighbourhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.6763574, -79.2930312.


In [145]:
limit=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=WCBNIU4Q5D3ZFWMLJO3EAYHIAKD2IKEJ0ZJDOJTWRN4EKOMW&client_secret=1GV4NPKTL2A1A3FSZYJFJS5EUBA3AXO0YXTT5Q0KBEPQRTXH&v=20180605&ll=43.6763574,-79.2930312&radius=500&limit=30'

In [170]:
#Saved the json manualy due to limited calls possible.
with open('toronto.json') as f:
    results = json.load(f)
results

{u'meta': {u'code': 200, u'requestId': u'5d402daa6e4650002c136d8b'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-4bd461bc77b29c74a07d9282-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          u'suffix': u'.png'},
         u'id': u'4bf58dd8d48988d159941735',
         u'name': u'Trail',
         u'pluralName': u'Trails',
         u'primary': True,
         u'shortName': u'Trail'}],
       u'id': u'4bd461bc77b29c74a07d9282',
       u'location': {u'address': u'Glen Manor',
        u'cc': u'CA',
        u'city': u'Toronto',
        u'country': u'Canada',
        u'crossStreet': u'Queen St.',
        u'distance': 89,
        u'formattedAddress': [u'Glen Manor (Queen St.)',
         u'Toronto ON',
         u'Canada'],
   

In [153]:
#Borrow category function from foursquare lab.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
print('Done !')

Done !


Converting the Json to pandas dataframe

In [154]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Park,Park,43.675278,-79.294647
4,Upper Beaches,Neighborhood,43.680563,-79.292869
5,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [113]:
nearby_venues.shape

(6, 4)

In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbuorhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [157]:
toronto_venues = getNearbyVenues(names=temp_df['Neighbourhood'],
                                   latitudes=temp_df['Latitude'],
                                   longitudes=temp_df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [158]:
print(toronto_venues.shape)
toronto_venues.head()

(1322, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [160]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,30,30,30,30,30,30


In [162]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


Analyzing each neighbourhood

In [164]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
toronto_onehot.shape

(1322, 236)

Grouping Row by neighbourhood ,by taking the mean of the frequency of occurrence of each category

In [167]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.033333,0.0,...,0.033333,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.0,0.055556,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.043478,0.0,...,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.033333,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000


Checking the new size

In [168]:
toronto_grouped.shape

(99, 236)

In [169]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0   Steakhouse  0.10
1  Coffee Shop  0.07
2  Pizza Place  0.07
3        Hotel  0.07
4         Café  0.07


----Agincourt----
               venue  freq
0     Breakfast Spot  0.25
1             Lounge  0.25
2     Clothing Store  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
               venue  freq
0         Playground   0.5
1               Park   0.5
2  Accessories Store   0.0
3      Movie Theater   0.0
4     Medical Center   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0          Pizza Place   0.2
1        Grocery Store   0.2
2  Fried Chicken Joint   0.1
3          Coffee Shop   0.1
4       Sandwich Place   0.1


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.22
1     Coffee Shop  0.11
2  Sandwich Place  0.11
3    Dance Studio  0

                        venue  freq
0  Construction & Landscaping  0.25
1                        Park  0.25
2            Basketball Court  0.25
3                      Bakery  0.25
4           Accessories Store  0.00


----East Birchmount Park,Ionview,Kennedy Park----
               venue  freq
0     Discount Store   0.2
1        Coffee Shop   0.2
2  Convenience Store   0.2
3   Department Store   0.2
4        Bus Station   0.2


----East Toronto----
               venue  freq
0        Pizza Place  0.25
1        Coffee Shop  0.25
2               Park  0.25
3  Convenience Store  0.25
4             Museum  0.00


----Emery,Humberlea----
               venue  freq
0     Baseball Field   1.0
1  Accessories Store   0.0
2   Malay Restaurant   0.0
3     Massage Studio   0.0
4     Medical Center   0.0


----Fairview,Henry Farm,Oriole----
            venue  freq
0  Clothing Store  0.13
1     Coffee Shop  0.10
2            Bank  0.03
3    Liquor Store  0.03
4          Bakery  0.03


----First Cana

               venue  freq
0         Playground   0.5
1      Jewelry Store   0.5
2  Accessories Store   0.0
3      Movie Theater   0.0
4     Medical Center   0.0


----Silver Hills,York Mills----
               venue  freq
0               Park   1.0
1  Accessories Store   0.0
2      Movie Theater   0.0
3     Massage Studio   0.0
4     Medical Center   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.13
1           Gastropub  0.10
2               Hotel  0.07
3          Restaurant  0.07
4  Italian Restaurant  0.07


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0                Café  0.10
1            Beer Bar  0.07
2        Cocktail Bar  0.07
3      Farmers Market  0.07
4  Seafood Restaurant  0.07


----Studio District----
                 venue  freq
0                 Café  0.13
1          Coffee Shop  0.10
2   Italian Restaurant  0.07
3  American Restaurant  0.07
4               Bakery  0.07


----The Annex,North Midtown,Yorkville

In [18]:
from math import radians, sin, cos, acos

print("Input coordinates of two points:")
slat = radians(float(input("Starting latitude: ")))
slon = radians(float(input("Ending longitude: ")))
elat = radians(float(input("Starting latitude: ")))
elon = radians(float(input("Ending longitude: ")))

dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
print("The distance is %.2fkm." % dist)

Input coordinates of two points:
Starting latitude: 12.9791198
Ending longitude: 77.5912997
Starting latitude: 55.245
Ending longitude: -162.759
The distance is 10583.51km.


In [25]:
from math import radians, sin, cos, acos

slat = radians(float(12.9791198))
slon = radians(float(77.5912997))
elat = radians(float(55.245))
elon = radians(float(-162.759))

dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
print("The distance is %.2fkm." % dist)

Input coordinates of two points:
The distance is 10583.51km.


In [14]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Bangalore')
latitude = location.latitude
longitude = location.longitude
print('{}- lat, {}- long'.format(latitude,longitude))

12.9791198- lat, 77.5912997- long
